Generating Figure 1 from GWTC-4 data

In [ ]:
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
from ligo.skymap.io.fits import read_sky_map
import glob
import re

skymap_files = glob.glob("/Users/leo/Desktop/gw-hands-on-school-2025/LVK_skyloc_samples/GWTC_skymaps/*Mixed_*.fits")

# Get all .fits files in directory
    
if not skymap_files:
    print(f"No .fits files found")
    
print(f"Found {len(skymap_files)} skymap files")
    
NSIDE = 128 
NPIX = hp.nside2npix(NSIDE)
    
    # Define the combined (added) skymap
combined_map = np.zeros(NPIX)
    
    # Process the skymaps
for i, fits_file in enumerate(skymap_files):
    try:
        # Read skymap
        skymap, header = read_sky_map(fits_file, nest=False, distances=False, moc=False)
            
        # Resample to common resolution(important with power=-2)
        skymap_resized = hp.ud_grade(skymap, NSIDE, pess=False, order_in='RING', order_out='RING', power=-2, dtype=None)
        
        # Add to combined map
        combined_map += skymap_resized / np.sum(skymap_resized)  # Normalize each map before adding, so that each contributes equally
            
        if (i+1) % 10 == 0:
            print(f"Processed {i+1}/{len(skymap_files)} files")
                
    except Exception as e:
        print(f"Error reading {fits_file}: {e}")
        continue
    

combined_map = combined_map / np.sum(combined_map)  # Normalize combined map

    
# Plot the combined skymap
plt.figure(figsize=(10, 6))
    
hp.mollview(
    combined_map,
    title=f"Combined GW Skymaps ({len(skymap_files)} events)",
    min=0,
    max=np.percentile(combined_map[combined_map > 0], 99),
    cbar=True,
    cmap='viridis',
    format='%.1e',
    unit="Probability Density $M_{GW}(\chi,\phi)$", 
    hold=True
    )
    
hp.graticule()
#plt.savefig(f"{event_name}-mollweide_skymap.png", format="png", dpi=300, transparent=True)
#Save combined skymap to(Requested by Donniel)
np.save('GW_combined_skymap.npy', combined_map)

Generating Figure 2

First generate a_{lm}

In [ ]:
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt

# USE the combined skymap from previous steps


lmax = 50  # Maximum ℓ to compute 
nside = hp.get_nside(combined_map)  # Get NSIDE from your map
print(f"Map NSIDE: {nside}")
print(f"Computing up to ℓ={lmax}")

# Compute spherical harmonic coefficients
alm = hp.map2alm(combined_map, lmax=lmax, iter=3)

print(f"Number of coefficients: {len(alm)}")
print(f"Shape of alm: {alm.shape}")

# Convert alm to list of (ℓ,m) coefficients
print("\nIndividual aₗₘ coefficients:")
for l in range(lmax+1):
    for m in range(-l, l+1):
        # Get index in alm array
        idx = hp.Alm.getidx(lmax, l, abs(m))
        
        # Get coefficient for positive and negative m (Negatative m using symmetry)
        if m < 0:
            a_lm = (-1)**m * np.conj(alm[idx])  # a_{l,-m} = (-1)^m a_{l,m}^*
        else:
            a_lm = alm[idx]
        
        #test
        if l <= 4:
            print(f"a_{{{l},{m}}} = {a_lm:.6e}  (|a| = {np.abs(a_lm):.6e})")

Get all the coefficients

In [ ]:
def extract_multipole(alm, lmax, target_l):
    """
    Extract all coefficients for a specific multipole ℓ
    """
    a_lm_list = []
    for m in range(-target_l, target_l+1):
        idx = hp.Alm.getidx(lmax, target_l, abs(m))
        if m < 0:
            a_lm = (-1)**m * np.conj(alm[idx])
        else:
            a_lm = alm[idx]
        a_lm_list.append((m, a_lm))
    
    return a_lm_list

# Extract test ℓ=0 to ℓ=4
print("=== Extracting Multipoles ℓ=0 to ℓ=4 ===")

for l in range(5):  # 0 to 4
    coeffs = extract_multipole(alm, lmax, l)
    
    print(f"\nℓ = {l}:")
    print("-" * 40)
    
    # For ℓ=0
    if l == 0:
        m, a_00 = coeffs[0]
        print(f"  Monopole (a_00) = {a_00:.6e}")
        print(f"  This is the AVERAGE")
        print(f"  Map average = {np.mean(combined_map):.6e}")
        
        # Check: a_00 should be √(4π) × average
        expected = np.sqrt(4*np.pi) * np.mean(combined_map)
        print(f"  Expected a_00 = √(4π)×mean = {expected:.6e}")
        print(f"  Difference: {np.abs(a_00 - expected)/np.abs(expected)*100:.2f}%")
    
    else:
        # Print all m
        for m, a_lm in coeffs:
            magnitude = np.abs(a_lm)
            phase = np.angle(a_lm)  # Phase in radians
            print(f"  m = {m:2d}: a_{{{l},{m}}} = {a_lm.real:+.6e} {a_lm.imag:+.6e}i")
            print(f"        |a| = {magnitude:.6e}, phase = {phase:.3f} rad")

Generate the spherical decomposition maps

In [ ]:
def reconstruct_from_ell_range(alm, lmax, ell_min, ell_max):
    """
    Reconstruct map using [ell_min, ell_max]
    """
    # Create a copy
    alm_filtered = alm.copy()
    
    # Zero out coefficients outside the desired ℓ range
    for l in range(lmax+1):
        if l < ell_min or l > ell_max:
            for m in range(-l, l+1):
                idx = hp.Alm.getidx(lmax, l, abs(m))
                alm_filtered[idx] = 0.0 + 0.0j
    
    # Reconstruct map
    map_reconstructed = hp.alm2map(alm_filtered, nside, lmax=lmax, verbose=False)
    
    return map_reconstructed, alm_filtered

# Store reconstructions
reconstructions = {}

"""TESTING SOME RANGES"""

# ℓ=0 (monopole)
map_ell0, alm_ell0 = reconstruct_from_ell_range(alm, lmax, 0, 0)
reconstructions['ell0'] = map_ell0

map_ell10, alm_ell10 = reconstruct_from_ell_range(alm, lmax, 0, 1)
reconstructions['ell10'] = map_ell10
# ℓ=1 (dipole)
map_ell1, alm_ell1 = reconstruct_from_ell_range(alm, lmax, 1, 1)
reconstructions['ell1'] = map_ell1


# ℓ=2 (quadrupole)
map_ell2, alm_ell2 = reconstruct_from_ell_range(alm, lmax, 2, 2)
reconstructions['ell2'] = map_ell2

# ℓ=3 (octupole)
map_ell3, alm_ell3 = reconstruct_from_ell_range(alm, lmax, 3, 3)
reconstructions['ell3'] = map_ell3  

# ℓ=4 
map_ell4, alm_ell4 = reconstruct_from_ell_range(alm, lmax, 4, 4)
reconstructions['ell4'] = map_ell4

# ℓ=5
map_ell5, alm_ell5 = reconstruct_from_ell_range(alm, lmax, 5, 5)
reconstructions['ell5'] = map_ell5

# ℓ=0-1 (monopole + dipole)
map_ell01, _ = reconstruct_from_ell_range(alm, lmax, 0, 1)
reconstructions['ell01'] = map_ell01

# ℓ=0-3 
map_ell03, alm_ell03 = reconstruct_from_ell_range(alm, lmax, 0, 3)
reconstructions['ell03'] = map_ell03  

# ℓ=0-4 (as requested)
map_ell04, _ = reconstruct_from_ell_range(alm, lmax, 0, 4)
reconstructions['ell04'] = map_ell04

# All ℓ up to lmax
map_all, _ = reconstruct_from_ell_range(alm, lmax, 0, lmax)
reconstructions['all'] = map_all


PLOTTING

In [ ]:
# Plot the reconstructions (MAYBE change later)
fig = plt.figure(figsize=(15, 12))


# ℓ=0 (monopole)
plt.subplot(3, 3, 1)
hp.mollview(map_ell0,
            title="ℓ=0 (Monopole)",
            min=0,
            max=np.percentile(map_ell0[map_ell0>0], 99) if np.any(map_ell0>0) else 1e-6,
            cbar=True,
            format='%.1e',
            hold=True,
            notext=True)
hp.graticule()

# ℓ=1 (dipole)
plt.subplot(3, 3, 2)
# Dipole can be negative, so we need symmetric color scale
vmax_ell1 = np.max(np.abs(map_ell1))
hp.mollview(map_ell1,
            title="ℓ=1 (Dipole)",
            min=-vmax_ell1,
            max=vmax_ell1,
            cmap='RdBu_r',  # Diverging colormap for positive/negative
            cbar=True,
            format='%.1e',
            hold=True,
            notext=True)
hp.graticule()

# ℓ=2 (quadrupole)
plt.subplot(3, 3, 3)
vmax_ell2 = np.max(np.abs(map_ell2))
hp.mollview(map_ell2,
            title="ℓ=2 (Quadrupole)",
            min=-vmax_ell2,
            max=vmax_ell2,
            cmap='RdBu_r',
            cbar=True,
            format='%.1e',
            hold=True,
            notext=True)
hp.graticule()



# ℓ=3
plt.subplot(3, 3, 4)
hp.mollview(map_ell3,
            title="ℓ=3 (Octopole)",
            min=0,
            max=np.percentile(map_ell3[map_ell3>0], 99),
            cbar=True,
            format='%.1e',
            hold=True,
            notext=True)
hp.graticule()

# ℓ=4
plt.subplot(3, 3, 5)
hp.mollview(map_ell4,
            title="ℓ=4",
            min=0,
            max=np.percentile(map_ell4[map_ell4>0], 99),
            cbar=True,
            format='%.1e',
            hold=True,
            notext=True)
hp.graticule()

# ℓ=5
plt.subplot(3, 3, 6)
hp.mollview(map_ell5,
            title="ℓ=5",
            min=0,
            max=np.percentile(map_ell5[map_ell5>0], 99),
            cbar=True,
            format='%.1e',
            hold=True,
            notext=True)
hp.graticule()

Angular Power Spectrum

In [ ]:
# Power spectrum
# Compute angular power spectrum C_ℓ
cl = hp.anafast(combined_map, lmax=lmax)
ell = np.arange(len(cl))



plt.plot(ell[1:], cl[1:], 'bo-', label='Observed GW')
plt.xlabel('ℓ')
plt.ylabel('$C_ℓ$')
plt.title('Angular Power Spectrum')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

Extra code for plotting spherically decomposed map by individual m values

In [ ]:
def plot_individual_m_modes(alm, lmax, target_l, nside):
    """
    Plot each m mode separately for a given ℓ
    """
    fig = plt.figure(figsize=(15, 10))
    
    # Create alm with only one m at a time
    for m_idx, m in enumerate(range(-target_l, target_l+1)):
        # Create empty alm
        alm_single = np.zeros_like(alm, dtype=complex)
        
        # Put only this (ℓ,m) coefficient
        idx = hp.Alm.getidx(lmax, target_l, abs(m))
        if m < 0:
            # Get the original a_{ℓ,|m|}
            a_lm_original = alm[idx]
            # For m<0: a_{ℓ,-m} = (-1)^m a_{ℓ,m}^*
            alm_single[idx] = (-1)**abs(m) * np.conj(a_lm_original)
        else:
            alm_single[idx] = alm[idx]
        
        # Also need the conjugate for negative m
        if m != 0:
            idx_conj = hp.Alm.getidx(lmax, target_l, abs(m))
            if m > 0:
                # For the -m mode when we have +m
                alm_single[idx_conj] = (-1)**m * np.conj(alm[idx])
        
        # Reconstruct map
        map_single = hp.alm2map(alm_single, nside, lmax=lmax, verbose=False)
        
        # Plot
        plt.subplot(2, 3, m_idx+1)  # 2 rows, 3 columns (5 plots + 1 for sum)
        
        vmax = np.max(np.abs(map_single))
        hp.mollview(map_single,
                   title=f"ℓ={target_l}, m={m}",
                   min=-vmax,
                   max=vmax,
                   cmap='RdBu_r',
                   cbar=True,
                   format='%.1e',
                   sub=(2, 3, m_idx+1),
                   hold=True,
                   notext=True)
        hp.graticule()
    
    # Plot the sum of all m modes (what you were plotting)
    plt.subplot(2, 3, 6)
    map_sum = hp.alm2map(alm, nside, lmax=lmax, verbose=False)
    
    # Test for only ℓ=2
    alm_ell2 = alm.copy()
    for l in range(lmax+1):
        if l != target_l:
            for mm in range(-l, l+1):
                idx = hp.Alm.getidx(lmax, l, abs(mm))
                alm_ell2[idx] = 0.0
    map_ell2_sum = hp.alm2map(alm_ell2, nside, lmax=lmax, verbose=False)
    
    vmax_sum = np.max(np.abs(map_ell2_sum))
    hp.mollview(map_ell2_sum,
               title=f"ℓ={target_l} (sum of all m)",
               min=-vmax_sum,
               max=vmax_sum,
               cmap='RdBu_r',
               cbar=True,
               format='%.1e',
               sub=(2, 3, 6),
               hold=True,
               notext=True)
    hp.graticule()
    
    plt.suptitle(f"Individual m modes for ℓ={target_l}", fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()
    
    return map_ell2_sum

# Plot ℓ=2 individual modes
map_ell2_sum = plot_individual_m_modes(alm, lmax, 2, nside)